In [2]:
import os
import time
#import faiss
import pickle
import gensim
import string
import argparse
import unicodedata

import numpy as np
import pandas as pd
import multiprocessing as mp

from collections import Counter
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from gensim import models, corpora
from gensim.models import LdaModel, nmf
from gensim.models.coherencemodel import CoherenceModel
from sklearn import metrics
from sklearn.mixture import GaussianMixture
from sklearn.feature_extraction.text import TfidfVectorizer

In [38]:
stopwords = [i for i in open("../data/lexicon/stopwords.txt")]
# def tokenize(s):
#     s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
#     s = ''.join(c for c in s if (c not in string.punctuation) or (c in (' ', '-', '_'))).lower()
#     t = RegexpTokenizer(r'\w+').tokenize(s)
#     t = [w.replace('-', '_').lower() for w in t if w not in stop_words and w != '' and w != '-']
#     return t

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import sent_tokenize, word_tokenize

def tokenizer(doc):
    return word_tokenize(doc)


In [27]:
def word_embedding_coherence(topics, glove_embeddings):
    c_we_per_topic = []
    c_we_std_per_topic = []

    for topic in topics:
        scores = []
        for i in range(len(topic)):
            for j in range(i+1, len(topic)):
                word1 = topic[i]
                word2 = topic[j]

                if word1 not in glove_embeddings or word2 not in glove_embeddings:
                    print(word1, 'or', word2, 'not in GloVe')
                    continue

                scores.append(glove_embeddings.similarity(word1, word2))

        c_we_per_topic.append(0 if len(scores) == 0 else np.mean(scores))
        c_we_std_per_topic.append(0 if len(scores) == 0 else np.std(scores))

    c_we = np.mean(c_we_per_topic)
    c_we_std = np.std(c_we_per_topic)

    return c_we, c_we_std, c_we_per_topic, c_we_std_per_topic

In [28]:
def get_related_words(word, thresh, vocabulary):
    try:
        neighborhood = pickle.load(open('/data/cache/neighborhoods/'+word+'.pickle', 'rb'))
    except:
        neighborhood = {word:{'sim': 1.0}}
    return word, {ww:neighborhood[ww]['sim'] for ww in neighborhood if ww in vocabulary and neighborhood[ww]['sim'] > thresh}

def get_related_vocabulary(word):
    return get_related_words(word, thresh=0, vocabulary=vocabulary)

In [29]:
# CONFIG
# USE KMEANS MODEL
# DEFINE # TOPICS = 200
# THRESHOLD = 0.15
# 10 Iterations
# Glove Path - TODOs

In [76]:
import json

args = [json.loads(ln)["argument"]["argument"] for ln in open("../data/cmv_processed.jsonl")]
args_df = pd.DataFrame(args[0:1000], columns=["argument"])

In [78]:
args_df
for i, j in args_df.iterrows():
    print(i, "j", )

0 j Section I Why is Basic Income Increasingly Popular? Basic income is a policy that has broad support from both the progressive left and libertarian right. Centerleft economists including Paul Krugman have endorsed the scheme for various reasons. First BI is an effective antipoverty measure. BI also reduces inequality by redistributing income from capital to labor.
1 j Section I Why is Basic Income Increasingly Popular? Basic income is a policy that has broad support from both the progressive left and libertarian right. Centerleft economists including Paul Krugman have endorsed the scheme for various reasons. First BI is an effective antipoverty measure. BI also reduces inequality by redistributing income from capital to labor.
2 j Section I Why is Basic Income Increasingly Popular? Basic income is a policy that has broad support from both the progressive left and libertarian right. Centerleft economists including Paul Krugman have endorsed the scheme for various reasons. First BI is

In [80]:
texts_raw = []

with tqdm(total=(len(args_df))) as pbar:
    for i in args_df.iterrows():
        texts_raw.append(tokenizer(j["argument"]))
        pbar.update()

  0%|          | 0/1000 [00:00<?, ?it/s]

In [81]:
texts_raw

[['People',
  'who',
  'say',
  'God',
  'has',
  'a',
  'plan',
  'for',
  'us',
  'all',
  'and',
  'things',
  'of',
  'this',
  'sort',
  'and',
  'who',
  'also',
  'believe',
  'that',
  'God',
  'has',
  'our',
  'best',
  'interests',
  'at',
  'heart',
  'should',
  'not',
  'complain',
  'about',
  'unfair',
  'circumstances',
  'or',
  'a',
  'negative',
  'event',
  'in',
  'their',
  'life',
  '.',
  'I',
  'think',
  'they',
  'are',
  'perfectly',
  'within',
  'their',
  'rights',
  'to',
  'grieve',
  'and',
  'feel',
  'bad.If',
  'someone',
  'believes',
  'that',
  'there',
  'is',
  'a',
  'set',
  'path',
  'that',
  'everyone',
  'follows',
  'in',
  'life',
  'that',
  'they',
  'can',
  'not',
  'change',
  'then',
  'whatever',
  'event',
  'occurs',
  'is',
  'undoubtedly',
  'a',
  'part',
  'of',
  'this',
  'predetermined',
  'fate',
  '.',
  'Whatever',
  'happens',
  'is',
  'all',
  'part',
  'of',
  'gods',
  'plan',
  'and',
  'that',
  'plan',
  'is'

In [82]:
dictionary = corpora.Dictionary(texts_raw)
vocabulary = sorted(dictionary.token2id.keys())

In [88]:
[i for i in vocabulary]

['.',
 'God',
 'I',
 'It',
 'People',
 'Whatever',
 'When',
 'a',
 'about',
 'all',
 'also',
 'an',
 'and',
 'are',
 'as',
 'at',
 'bad.If',
 'be',
 'being',
 'believe',
 'believes',
 'best',
 'can',
 'challenge',
 'change',
 'circumstances',
 'comfort',
 'complain',
 'detrimental',
 'event',
 'everyone',
 'fate',
 'feel',
 'feelings',
 'follows',
 'for',
 'given',
 'god',
 'gods',
 'good',
 'greatest',
 'grieve',
 'hand',
 'happens',
 'has',
 'heart',
 'important',
 'in',
 'incredibly',
 'instead',
 'interests',
 'is',
 'it',
 'life',
 'look',
 'more',
 'negative',
 'not',
 'occurs',
 'of',
 'or',
 'our',
 'overcome',
 'part',
 'path',
 'perfectly',
 'plan',
 'planned',
 'predetermined',
 'punishing',
 'rights',
 'say',
 'selfish',
 'set',
 'should',
 'someone',
 'something',
 'sort',
 'than',
 'that',
 'the',
 'their',
 'them',
 'then',
 'there',
 'they',
 'things',
 'think',
 'this',
 'to',
 'undoubtedly',
 'unfair',
 'us',
 'what',
 'whatever',
 'who',
 'within',
 'would']

In [91]:
# Create Vocab
words_and_related_vocab = [get_related_vocabulary(i) for i in vocabulary]
related_vocab = dict(words_and_related_vocab)

In [92]:
words_and_related_vocab

[('.', {'.': 1.0}),
 ('God', {'God': 1.0}),
 ('I', {'I': 1.0}),
 ('It', {'It': 1.0}),
 ('People', {'People': 1.0}),
 ('Whatever', {'Whatever': 1.0}),
 ('When', {'When': 1.0}),
 ('a', {'a': 1.0}),
 ('about', {'about': 1.0}),
 ('all', {'all': 1.0}),
 ('also', {'also': 1.0}),
 ('an', {'an': 1.0}),
 ('and', {'and': 1.0}),
 ('are', {'are': 1.0}),
 ('as', {'as': 1.0}),
 ('at', {'at': 1.0}),
 ('bad.If', {'bad.If': 1.0}),
 ('be', {'be': 1.0}),
 ('being', {'being': 1.0}),
 ('believe', {'believe': 1.0}),
 ('believes', {'believes': 1.0}),
 ('best', {'best': 1.0}),
 ('can', {'can': 1.0}),
 ('challenge', {'challenge': 1.0}),
 ('change', {'change': 1.0}),
 ('circumstances', {'circumstances': 1.0}),
 ('comfort', {'comfort': 1.0}),
 ('complain', {'complain': 1.0}),
 ('detrimental', {'detrimental': 1.0}),
 ('event', {'event': 1.0}),
 ('everyone', {'everyone': 1.0}),
 ('fate', {'fate': 1.0}),
 ('feel', {'feel': 1.0}),
 ('feelings', {'feelings': 1.0}),
 ('follows', {'follows': 1.0}),
 ('for', {'for': 1.0

In [93]:
related_vocab

{'.': {'.': 1.0},
 'God': {'God': 1.0},
 'I': {'I': 1.0},
 'It': {'It': 1.0},
 'People': {'People': 1.0},
 'Whatever': {'Whatever': 1.0},
 'When': {'When': 1.0},
 'a': {'a': 1.0},
 'about': {'about': 1.0},
 'all': {'all': 1.0},
 'also': {'also': 1.0},
 'an': {'an': 1.0},
 'and': {'and': 1.0},
 'are': {'are': 1.0},
 'as': {'as': 1.0},
 'at': {'at': 1.0},
 'bad.If': {'bad.If': 1.0},
 'be': {'be': 1.0},
 'being': {'being': 1.0},
 'believe': {'believe': 1.0},
 'believes': {'believes': 1.0},
 'best': {'best': 1.0},
 'can': {'can': 1.0},
 'challenge': {'challenge': 1.0},
 'change': {'change': 1.0},
 'circumstances': {'circumstances': 1.0},
 'comfort': {'comfort': 1.0},
 'complain': {'complain': 1.0},
 'detrimental': {'detrimental': 1.0},
 'event': {'event': 1.0},
 'everyone': {'everyone': 1.0},
 'fate': {'fate': 1.0},
 'feel': {'feel': 1.0},
 'feelings': {'feelings': 1.0},
 'follows': {'follows': 1.0},
 'for': {'for': 1.0},
 'given': {'given': 1.0},
 'god': {'god': 1.0},
 'gods': {'gods': 1.

In [94]:
cache_path = "cache"
dataset_name="test"

pickle.dump(related_vocab, open(cache_path+dataset_name+'_related_vocab.pickle', 'wb'))
print('Related vocabulary saved into cache.')

Related vocabulary saved into cache.


In [101]:
thresh = 0.15
related_vocab_path = cache_path+dataset_name+'_related_vocab_thresh_'+str(thresh)+'.pickle'

In [102]:
print("processing related vocabulary for threshold", thresh)

processing related vocabulary for threshold 0.15


In [103]:
related_vocab_thresh = {}
for word in tqdm(related_vocab):
    related_vocab_thresh[word] = {w:related_vocab[word][w] for w in related_vocab[word]
        if w in vocabulary and related_vocab[word][w] > thresh}

  0%|          | 0/98 [00:00<?, ?it/s]

In [105]:
pickle.dump(related_vocab_thresh, open(related_vocab_path, 'wb'))
print('Related thresh ('+str(thresh)+') vocabulary saved into cache.')

Related thresh (0.15) vocabulary saved into cache.


In [106]:
related_vocab = pickle.load(open(related_vocab_path, 'rb'))

In [107]:
related_vocab

{'.': {'.': 1.0},
 'God': {'God': 1.0},
 'I': {'I': 1.0},
 'It': {'It': 1.0},
 'People': {'People': 1.0},
 'Whatever': {'Whatever': 1.0},
 'When': {'When': 1.0},
 'a': {'a': 1.0},
 'about': {'about': 1.0},
 'all': {'all': 1.0},
 'also': {'also': 1.0},
 'an': {'an': 1.0},
 'and': {'and': 1.0},
 'are': {'are': 1.0},
 'as': {'as': 1.0},
 'at': {'at': 1.0},
 'bad.If': {'bad.If': 1.0},
 'be': {'be': 1.0},
 'being': {'being': 1.0},
 'believe': {'believe': 1.0},
 'believes': {'believes': 1.0},
 'best': {'best': 1.0},
 'can': {'can': 1.0},
 'challenge': {'challenge': 1.0},
 'change': {'change': 1.0},
 'circumstances': {'circumstances': 1.0},
 'comfort': {'comfort': 1.0},
 'complain': {'complain': 1.0},
 'detrimental': {'detrimental': 1.0},
 'event': {'event': 1.0},
 'everyone': {'everyone': 1.0},
 'fate': {'fate': 1.0},
 'feel': {'feel': 1.0},
 'feelings': {'feelings': 1.0},
 'follows': {'follows': 1.0},
 'for': {'for': 1.0},
 'given': {'given': 1.0},
 'god': {'god': 1.0},
 'gods': {'gods': 1.

In [ ]:
# Experiment Code
# os.system('mkdir results/'+experiment_id)
# os.chmod('results/'+experiment_id, 0o777)

In [115]:
# K-Means
import faiss
n_topics = 10
iterations = 5
random_state = 27

corpus_cs = []
for d in texts_raw:
    original_doc_words = d
    final_doc = []
    for word in original_doc_words:
        final_doc.extend([word] if word not in related_vocab else list(related_vocab[word].keys()))
    corpus_cs.append(final_doc)

time_to_train = time.time()
vectorizer = TfidfVectorizer(min_df=2, stop_words='english')
X = vectorizer.fit_transform([' '.join(d) for d in corpus_cs]).toarray()
id2word = {w:i for i,w in vectorizer.vocabulary_.items()}

kmeans = faiss.Kmeans(d=X.shape[1], k=n_topics, niter=iterations, nredo=1, seed=random_state)
kmeans.train(X.astype(np.float32))
time_to_train = time.time() - time_to_train

topic_preds = kmeans.index.search(X.astype(np.float32), 1)[1].reshape((X.shape[0], ))
time_to_preds = time.time() - time_to_train

cluster_centers = kmeans.centroids
topic_words = []
for cluster in cluster_centers:
    sorted_values = sorted(list(enumerate(cluster)), key=lambda x: -x[1])
    top_words = [(id2word[w[0]], w[1]) for w in sorted_values[:10]]
    topic_words.append([w[0] for w in top_words])

In [116]:
topic_words

[['god',
  'life',
  'plan',
  'event',
  'feel',
  'happens',
  'unfair',
  'believe',
  'best',
  'change'],
 ['god',
  'life',
  'plan',
  'event',
  'feel',
  'happens',
  'unfair',
  'bad',
  'believes',
  'challenge'],
 ['god',
  'life',
  'plan',
  'event',
  'feel',
  'happens',
  'unfair',
  'believe',
  'best',
  'change'],
 ['god',
  'life',
  'plan',
  'event',
  'feel',
  'happens',
  'unfair',
  'believe',
  'best',
  'change'],
 ['god',
  'life',
  'plan',
  'event',
  'feel',
  'happens',
  'unfair',
  'bad',
  'believe',
  'believes'],
 ['god',
  'life',
  'plan',
  'event',
  'feel',
  'happens',
  'unfair',
  'bad',
  'believes',
  'challenge'],
 ['god',
  'life',
  'plan',
  'event',
  'feel',
  'happens',
  'unfair',
  'bad',
  'believes',
  'challenge'],
 ['god',
  'life',
  'plan',
  'event',
  'feel',
  'happens',
  'unfair',
  'bad',
  'believe',
  'believes'],
 ['god',
  'life',
  'plan',
  'event',
  'feel',
  'happens',
  'unfair',
  'believe',
  'best',
  '

In [ ]:
run = {}
run['args'] = args
run['topics'] = topic_words
run['times'] = {'train': time_to_train, 'pred': time_to_preds, 'eval':time_to_eval}
run['metrics'] = {'completeness': completeness,
                  'homogeneity': homogeneity,
                  'v_measure':v_measure,
                  'coherence':coherence,
                  'wb_coherence': (c_we, c_we_std, c_we_per_topic, c_we_std_per_topic)}

pickle.dump(run, open(args['save_path'] + experiment_id + '/results.pickle', 'wb'))

print('RUN', random_state)
print(run)